In [1]:
import anndata as ad
import scanpy as sc
import numpy as np
import scipy as scp
import os
import re

In [2]:
#Read the downloaded output file names and combine file with same S
adataNames = os.listdir('./adataObjsKeep/')
adataList = []
adataNames.sort()
adataNames

['S13_L002_adata_keep.h5ad',
 'S13_L003_adata_keep.h5ad',
 'S13_L004_adata_keep.h5ad',
 'S13_L005_adata_keep.h5ad',
 'S13_L006_adata_keep.h5ad',
 'S13_L007_adata_keep.h5ad',
 'S13_L008_adata_keep.h5ad',
 'S14_L002_adata_keep.h5ad',
 'S14_L003_adata_keep.h5ad',
 'S14_L004_adata_keep.h5ad',
 'S14_L005_adata_keep.h5ad',
 'S14_L006_adata_keep.h5ad',
 'S14_L007_adata_keep.h5ad',
 'S14_L008_adata_keep.h5ad',
 'S15_L002_adata_keep.h5ad',
 'S15_L003_adata_keep.h5ad',
 'S15_L004_adata_keep.h5ad',
 'S15_L005_adata_keep.h5ad',
 'S15_L006_adata_keep.h5ad',
 'S15_L007_adata_keep.h5ad',
 'S15_L008_adata_keep.h5ad',
 'S16_L002_adata_keep.h5ad',
 'S16_L003_adata_keep.h5ad',
 'S16_L004_adata_keep.h5ad',
 'S16_L005_adata_keep.h5ad',
 'S16_L006_adata_keep.h5ad',
 'S16_L007_adata_keep.h5ad',
 'S16_L008_adata_keep.h5ad',
 'S17_L002_adata_keep.h5ad',
 'S17_L003_adata_keep.h5ad',
 'S17_L004_adata_keep.h5ad',
 'S17_L005_adata_keep.h5ad',
 'S17_L006_adata_keep.h5ad',
 'S17_L007_adata_keep.h5ad',
 'S17_L008_ada

In [3]:
# Save same sample indices in a dictionary
samples = []
sampleDict = {}
for i,an in enumerate(adataNames):
    match = re.search('S[0-9]+',an).group()
    samples.append(match)
    if match in sampleDict:
        sampleDict[match].append(i)
    else:
        sampleDict[match] = [i]
sampleDict

{'S13': [0, 1, 2, 3, 4, 5, 6],
 'S14': [7, 8, 9, 10, 11, 12, 13],
 'S15': [14, 15, 16, 17, 18, 19, 20],
 'S16': [21, 22, 23, 24, 25, 26, 27],
 'S17': [28, 29, 30, 31, 32, 33, 34],
 'S18': [35, 36, 37, 38, 39, 40, 41],
 'S1': [42, 43, 44, 45, 46, 47, 48, 49],
 'S2': [50, 51, 52, 53, 54, 55, 56],
 'S3': [57, 58, 59, 60, 61, 62, 63],
 'S4': [64, 65]}

In [4]:
list(sampleDict.keys())

['S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6']

In [ ]:
#The idea is to loop through each sample and then join them together
for s in list(sampleDict.keys())[:5]:
# for s in sampleDict.keys()
    indices = sampleDict[s]
    len_indices = len(indices)
    mid = int(len_indices/2)
    indxs = [indices[:mid],indices[mid:]]
    for j in range(2):
        print('working on iteration ' + str(j+1) + '/2 for ' + s )
        indx = indxs[j]
        adataList = []
        for i in indx:
            adataList.append(ad.read_h5ad('./adataObjs/'+adataNames[i]))
        print(adataList)
        checkSameVar = []
        for i in range(1,len(adataList)):
            checkSameVar.append(all(adataList[i].var.index == adataList[i-1].var.index))
        check = all(checkSameVar)
        assert check == True, "The vars are not same in all adata"
        #convert each sparse matrix to array
        XList = []
        SList = []
        UList = []
        obs = []
        var = []
        for an in adataList:
            XList.append(an.X.toarray())
            SList.append(an.layers['spliced'].toarray())
            UList.append(an.layers['unspliced'].toarray())
            obs.append(an.obs)
            var.append(an.var)
        cellDict = {}
        X = np.array([XList[0][0]])
        U = np.array([SList[0][0]])
        S = np.array([UList[0][0]])
        obs_names = []
        #check if var names are same in all objects in adataList
        var_names = adataList[0].var.index
        obs_names.append(adataList[0].obs.index[0])

        cellDict[adataList[0].obs.index[0]] = 0
        nextIndex = 1 #next index of final X,S, and U

        for i in range(len(adataList)):
            if i == 0:
                for j in range(1,adataList[i].shape[0]):
                    if adataList[i].obs.index[j] in cellDict: # existing barcode
                        idx = cellDict[obs[i].index[j]]
                        X[idx] = X[idx] + XList[i][j]
                        S[idx] = S[idx] + SList[i][j]
                        U[idx] = U[idx] + UList[i][j]
                    else: # new barcode
                        cellDict[obs[i].index[j]] = nextIndex
                        nextIndex += 1
                        X = np.append(X,[XList[i][j]],axis=0)
                        S = np.append(S,[SList[i][j]],axis=0)
                        U = np.append(U,[UList[i][j]],axis=0)
                        obs_names.append(obs[i].index[j])
            else:
                for j in range(adataList[i].shape[0]):
                    if adataList[i].obs.index[j] in cellDict: # existing barcode
                        idx = cellDict[obs[i].index[j]]
                        X[idx] = X[idx] + XList[i][j]
                        S[idx] = S[idx] + SList[i][j]
                        U[idx] = U[idx] + UList[i][j]
                    else: # new barcode
                        cellDict[obs[i].index[j]] = nextIndex
                        nextIndex += 1
                        X = np.append(X,[XList[i][j]],axis=0)
                        S = np.append(S,[SList[i][j]],axis=0)
                        U = np.append(U,[UList[i][j]],axis=0)
                        obs_names.append(obs[i].index[j])
        X = scp.sparse.csr_matrix(X)
        S = scp.sparse.csr_matrix(S)
        U = scp.sparse.csr_matrix(U)
        concatAdata = ad.AnnData(X,
                      {'obs_names': obs_names},
                      {'var_names': var.values},
                       layers={'spliced':S,
                              'unspliced':U})
        filename = s + '_' + str(j+1) + '.h5ad'
        print('joint adata is:')
        print(concatAdata25)
        print('writing file '+filename)
        concatAdata.write(filename)

[AnnData object with n_obs × n_vars = 154041 × 60623 
    layers: 'spliced', 'unspliced', AnnData object with n_obs × n_vars = 118165 × 60623 
    layers: 'spliced', 'unspliced', AnnData object with n_obs × n_vars = 157825 × 60623 
    layers: 'spliced', 'unspliced']


In [6]:
X

array([<1x60623 sparse matrix of type '<class 'numpy.float32'>'
	with 1 stored elements in Compressed Sparse Row format>], dtype=object)

In [ ]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)